In [1]:
import csv

class Victim:
    indexMapping = {"year":1,"month":2,"intent":3,"policy":4,"sex":5,"age":6,"race":7,"hispanic":8,"place":9,"education":10}
    
    def __init__(self,row):
        self.row = row
        
    def getValue(self,columnName):
        #print(self.row[0])
        return self.row[self.indexMapping[columnName]]
    
    def getIndex(self,columnName):
        return self.indexMapping[columnName]
      
        
class Census:
    
    indexMapping = {"White":10,"Hispanic":11,"Black":12,"Native American/Native Alaskan":13,"Asian":14,"Native Asian":15}
    popDict = {}
    
    def __init__(self,row):
        self.row = row
        self.popDict["Asian/Pacific Islander"] = int(self.getValue("Asian"))+int(self.getValue("Native Asian"))
        self.popDict["Black"] = int(self.getValue("Black"))
        self.popDict["Hispanic"] = int(self.getValue("Hispanic"))
        self.popDict["Native American/Native Alaskan"] = int(self.getValue("Native American/Native Alaskan"))
        self.popDict["White"] = int(self.getValue("White"))
        
    def getValue(self,columnName):
        return self.row[self.indexMapping[columnName]]
    
    def getIndex(self,columnName):
        return self.indexMapping[columnName]


In [2]:
class GdReport:
    victimList = []
    censusList = []
    
    def __init__(self,guns_file,census_file):
        self.convertGunsToObject(open(guns_file,"r"),"guns")
        self.convertGunsToObject(open(census_file,"r"),"census")
        
    def convertGunsToObject(self, inputFile, objectType):
        csvReader = csv.reader(inputFile)
        rows = list(csvReader)
        
        for row in rows[1:]:
            if objectType == "guns" :
                victim = Victim(row)
                self.victimList.append(victim)
            elif objectType == "census":
                census = Census(row)
                self.censusList.append(census)
                
    def calRate(self,primaryGroupName,primaryGroupValue,secondaryGroupName):
        
        result = {}
        
        for victim in self.victimList:
            if victim.getValue(primaryGroupName) == primaryGroupValue:
                if victim.getValue(secondaryGroupName) not in result:
                    result[victim.getValue(secondaryGroupName)] = 1
                else:
                    result[victim.getValue(secondaryGroupName)] += 1
        
        return result
            
    def calHundredRaceRate(self,intentValue):
        result = self.calRate("intent",intentValue,"race")
        
        for key,value in result.items():
            print(key)
            print(self.censusList[0].popDict[key])
            print(value)
            result[key] = value/int(self.censusList[0].popDict[key])*100000
            
        return result

report = GdReport("guns.csv","census.csv") 

Read me:
call calRate to get counts distribution:
1st parameter : column to filter e.g intent
2nd parameter : value to filter e.g Suicide
3rd parameter : colomn to be check for distribution e.g sex

call calHundredRaceRate to get counts with the consideration of racial ratio
1st parameter : intent value

refer to the following for examples

In [10]:
homicide_race_counts  = report.calHundredRaceRate("Homicide")
print(homicide_race_counts )

print(report.calRate("intent","Homicide","race"))

White
197318956
9147
Hispanic
44618105
5634
Black
40250635
19510
Native American/Native Alaskan
3739506
326
Asian/Pacific Islander
15834141
559
{'White': 4.6356417981453335, 'Hispanic': 12.627161104219914, 'Black': 48.471284987180944, 'Native American/Native Alaskan': 8.717729026240365, 'Asian/Pacific Islander': 3.530346230970155}
{'White': 9147, 'Hispanic': 5634, 'Black': 19510, 'Native American/Native Alaskan': 326, 'Asian/Pacific Islander': 559}


(1) homicide rate by race
Race of Black victim has the most homicide intent among all other race.

In [6]:
result = report.calRate("intent","Homicide","sex")
print(result)

{'F': 10746, 'M': 59606}


(2) Homicide rate by sex
Male victim is almost 6 times higher the Female

In [7]:
result = report.calRate("intent","Homicide","education")
print(result)

{'5': 910, '3': 11280, 'NA': 68, '2': 31298, '4': 3118, '1': 23678}


(3) Homocide rate by education level
education level is related, the lower the education the lower the victim rate

In [15]:
result = report.calRate("race","White","intent")
print(result)

{'NA': 6, 'Homicide': 54882, 'Accidental': 6792, 'Undetermined': 3510, 'Suicide': 332232}


(4) intent distribution for whice race
suicide is the highest among all the intent

In [14]:
result = report.calRate("race","Black","intent")
print(result)

{'Undetermined': 126, 'Accidental': 328, 'Homicide': 19510, 'Suicide': 3332}


(5) intent distribution amont black race
Homicide is the highest among all the intent

In [15]:
result = report.calRate("intent","Homicide","place")
print(result)

{'Residential institution': 73, 'School/instiution': 361, 'Sports': 22, 'Street': 8867, 'NA': 1383, 'Other specified': 5429, 'Industrial/construction': 85, 'Other unspecified': 3604, 'Home': 13613, 'Farm': 76, 'Trade/service area': 1663}
